# Train

In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')

PREFIX = '../../input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/detectors_cascade_rcnn_r50_1x_trash'

for bbox_head in cfg.model.roi_head.bbox_head:
    bbox_head.num_classes = 11
print(cfg.model)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

{'type': 'CascadeRCNN', 'pretrained': 'torchvision://resnet50', 'backbone': {'type': 'DetectoRS_ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'conv_cfg': {'type': 'ConvAWS'}, 'sac': {'type': 'SAC', 'use_deform': True}, 'stage_with_sac': (False, True, True, True), 'output_img': True}, 'neck': {'type': 'RFP', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5, 'rfp_steps': 2, 'aspp_out_channels': 64, 'aspp_dilations': (1, 3, 6, 1), 'rfp_backbone': {'rfp_inplanes': 256, 'type': 'DetectoRS_ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'conv_cfg': {'type': 'ConvAWS'}, 'sac': {'type': 'SAC', 'use_deform': True}, 'stage_with_sac': (False, True, True, True), 'pretrained': 'torchvision://resnet50', 'style': 'pytorch'}}, 'rpn_head': 

In [3]:
model = build_detector(cfg.model)

2021-05-11 05:56:31,901 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 05:56:31,902 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 05:56:32,199 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.39s)
creating index...
index created!


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-11 05:56:40,509 - mmdet - INFO - Start running, host: root@a1b075ba2232, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/detectors_cascade_rcnn_r50_1x_trash
2021-05-11 05:56:40,511 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs


Done (t=0.82s)
creating index...
index created!


2021-05-11 05:57:44,636 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 2:46:54, time: 1.282, data_time: 0.051, memory: 5799, loss_rpn_cls: 0.4593, loss_rpn_bbox: 0.0721, s0.loss_cls: 0.8066, s0.acc: 83.5938, s0.loss_bbox: 0.1802, s1.loss_cls: 0.3463, s1.acc: 84.3066, s1.loss_bbox: 0.0513, s2.loss_cls: 0.1569, s2.acc: 88.9326, s2.loss_bbox: 0.0070, loss: 2.0797, grad_norm: 11.1057
2021-05-11 05:58:46,712 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 2:43:12, time: 1.242, data_time: 0.007, memory: 5799, loss_rpn_cls: 0.2367, loss_rpn_bbox: 0.0767, s0.loss_cls: 0.4818, s0.acc: 89.4609, s0.loss_bbox: 0.2486, s1.loss_cls: 0.1368, s1.acc: 95.3691, s1.loss_bbox: 0.0711, s2.loss_cls: 0.0427, s2.acc: 97.5713, s2.loss_bbox: 0.0108, loss: 1.3052, grad_norm: 5.2679
2021-05-11 05:59:48,877 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-03, eta: 2:41:21, time: 1.243, data_time: 0.006, memory: 5799, loss_rpn_cls: 0.2117, loss_rpn_bbox: 0.0636, s0.loss_cls: 0.4694, s0.acc: 89.2578, s

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 7.0 task/s, elapsed: 93s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 6.7 task/s, elapsed: 98s, ETA:     0s

2021-05-11 09:00:25,540 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.88s).
Accumulating evaluation results...


2021-05-11 09:00:31,207 - mmdet - INFO - Epoch(val) [12][655]	bbox_mAP: 0.2230, bbox_mAP_50: 0.3390, bbox_mAP_75: 0.2220, bbox_mAP_s: 0.0420, bbox_mAP_m: 0.1950, bbox_mAP_l: 0.4290, bbox_mAP_copypaste: 0.223 0.339 0.222 0.042 0.195 0.429


DONE (t=0.65s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.223
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.339
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.222
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.042
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.195
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.429
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.362
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.362
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.362
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.108
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.316
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.648


# Test

In [6]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [7]:
cfg.model.train_cfg = None
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [8]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-11 09:00:34,107 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 09:00:34,108 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 09:00:34,433 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

Use load_from_local loader


In [10]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 7.0 task/s, elapsed: 119s, ETA:     0s

In [11]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.39s)
creating index...
index created!


,PredictionString,image_id
0,1 0.7318308 140.22397 51.37345 259.66284 172.4...,batch_01_vt/0021.jpg
1,1 0.20228997 1.6392059 245.8529 300.34222 457....,batch_01_vt/0028.jpg
2,0 0.05953554 0.0 395.39032 0.0 429.68085 1 0.9...,batch_01_vt/0031.jpg
3,1 0.2111828 0.0 221.50827 36.351246 311.7393 8...,batch_01_vt/0032.jpg
4,1 0.82041365 388.36865 264.85236 487.73004 345...,batch_01_vt/0070.jpg
